## Phase 1 Pipeline

In [1]:
from model_training import *

In [2]:
from convolutional_nn import *
from data_processing import *
import pandas as pd

Using TensorFlow backend.


In [3]:
sensors = ['leftJointPosition', 'rightJointPosition', 'leftJointVelocity',
           'rightJointVelocity', 'imuGyroX', 'imuGyroY', 'imuGyroZ', 'imuAccX',
           'imuAccY', 'imuAccZ']

# Produce 1 label file for each trial and store them in ../labels folder
data = import_data(sensors)

left_joint_positions, right_joint_positions = extract_joint_positions(data)
labels = []
for i in range(5):
    filename = "labels/label_trial{}.txt".format(i+1)
    left_gp_x, left_gp_y = label_vectors(left_joint_positions[i])
    right_gp_x, right_gp_y = label_vectors(right_joint_positions[i])
    labeled_gp = pd.DataFrame({'leftGaitPhase_x': left_gp_x, 'leftGaitPhase_y': left_gp_y,
                               'rightGaitPhase_x': right_gp_x, 'rightGaitPhase_y': right_gp_y})
    labels.append(labeled_gp)

# Combine the data and the labels
for d, l in zip(data, labels):
    d[l.columns] = l

# Creat a list of cut_indicies for each trial
cut_indicies_list = []
for i in range(5):
    cut_indicies_list.append(find_cutting_indices(left_joint_positions[i], 
    right_joint_positions[i]))

# Cut the standing data and store files into ../features folder
data_list = cnn_cut_data(data, cut_indicies_list)

In [4]:
hyperparam_space = {
    'window_size': [20],
    'model': 'lstm',
    'lstm': {
      'units': [10],
      'activation': ['relu']
    },
    'dense': {
        'activation': ['tanh']
    },
    'optimizer': {
        'loss': ['mean_absolute_error'],
        'optimizer': ['adam']
    },
    'training': {
        'epochs': [2],
        'batch_size': [128]
    }
}

In [5]:
hyperparameter_configs = get_model_configs(hyperparam_space)
print(hyperparameter_configs)

[{'window_size': 20, 'model': 'lstm', 'dense': {'activation': 'tanh'}, 'optimizer': {'loss': 'mean_absolute_error', 'optimizer': 'adam'}, 'training': {'epochs': 2, 'batch_size': 128}, 'lstm': {'units': 10, 'activation': 'relu'}}]


In [6]:
trial_results, average_results = train_models('lstm', hyperparameter_configs, data_list)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 20, 10)            21        
_________________________________________________________________
lstm (LSTM)                  (None, 10)                840       
_________________________________________________________________
dense (Dense)                (None, 4)                 44        
Total params: 905
Trainable params: 884
Non-trainable params: 21
_________________________________________________________________
Train on 28216 samples, validate on 7055 samples
Epoch 1/2
28216/28216 [==============================] - 6s 221us/sample - loss: 0.3312 - val_loss: 0.1532
Epoch 2/2
28216/28216 [==============================] - 3s 110us/sample - loss: 0.1082 - val_loss: 0.0998
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Sh

In [7]:
display(trial_results)

,window_size,lstm_units,lstm_activation,dense_activation,optim_loss,optim_optimizer,training_epochs,training_batch_size,left_validation_rmse,right_validation_rmse,trial
0,20,10,relu,tanh,mean_absolute_error,adam,2,128,2.437452,2.211019,0
1,20,10,relu,tanh,mean_absolute_error,adam,2,128,3.524134,3.693792,1
